## ROUGH TESTING FOR QUANTUM SIMULATOR

In [47]:
import numpy as np
from numpy.random import choice

initialize state

read program, and for each gate:

    calculate matrix operator
    
    apply operator (modify state)

perform multi-shot measurement of all qubits using weighted random technique

In [114]:
single_q_gates = {'x':np.array([[0, 1], [1, 0]]),
                  'h':np.array([[1/np.sqrt(2), 1/np.sqrt(2)], [1/np.sqrt(2), -1/np.sqrt(2)]])}
multi_q_gates = {'cx':np.array([[0, 1], [1, 0]])}

I = np.identity(2)
P0X0 = np.array([[1, 0],[0, 0]])
P1X1 = np.array([[0, 0],[0, 1]])

In [129]:
bin_i = [int(x) for x in bin(3)[2:].zfill(3)]
print(bin_i)

[0, 1, 1]


In [115]:
def get_ground_state(num_qubits):
    psi = np.zeros_like(np.arange(2**num_qubits))
    psi[0] = 1
    return psi

In [116]:
def get_operator(total_qubits, gate_unitary, target_qubits):
    if gate_unitary in single_q_gates:
        target = target_qubits[0]
        operator = 1
        for i in range(total_qubits):
            if i == target_qubits[0]:
                operator = np.kron(operator, single_q_gates[gate_unitary])
            else:
                operator = np.kron(operator, I)
        return operator
    elif gate_unitary in multi_q_gates:
        control = target_qubits[0]
        target = target_qubits[1]
        operator_a = 1
        operator_b = 1
        for i in range(total_qubits):
            if i == control:
                operator_a = np.kron(operator_a, P0X0)
                operator_b = np.kron(operator_b, P1X1)
            elif i == target:
                operator_a = np.kron(operator_a, I)
                operator_b = np.kron(operator_b, multi_q_gates[gate_unitary])
            else:
                operator_a = np.kron(operator_a, I)
                operator_b = np.kron(operator_b, I)
        return operator_a + operator_b

In [117]:
def run_program(initial_state, program):
    num_qubits = len(initial_state).bit_length()-1
    state = initial_state
    for operation in program:
        gate, target = operation['gate'], operation['target']
        operator = get_operator(num_qubits, gate, target)
        state = np.dot(operator, state)
    return state

In [118]:
my_circuit = [
{ "gate": "x", "target": [0] },
{ 'gate': 'cx', 'target': [0,2] }
]
initial_state = get_ground_state(4)
state = run_program(initial_state, my_circuit)
print(state)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [119]:
def measure_all(state_vector):
    return "{0:b}".format(choice(range(len(state_vector)), 1, p=np.square(state_vector))[0]).zfill(len(state_vector).bit_length()-1)

In [120]:
def get_counts(state_vector, num_shots):
    counts = {}
    for i in range(num_shots):
        measurement = measure_all(state_vector)
        if measurement not in counts.keys():
            counts[measurement] = 1
        else:
            counts[measurement] += 1 
    return counts

In [121]:
print(get_counts(state, 1000))

{'1010': 1000}


In [126]:
class var:
    a = 5

class test:
    def __init__(self,b):
        self.b = b
    def m(self):
        print(self.b*var.a)

In [127]:
A = test(4)
A.m()

20
